In [1]:
!git clone https://github.com/SIDN-IAP/global-model-repr
!pip install transformers

import torch
from transformers import BertTokenizer, BertModel
import sys
sys.path.append('global-model-repr/')
from probing.utils import get_sentence_repr, get_model_and_tokenizer, get_pos_data

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print("device:", device)

Cloning into 'global-model-repr'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 176 (delta 57), reused 147 (delta 36), pack-reused 0
Receiving objects: 100% (176/176), 1.15 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (57/57), done.
     |████████████████████████████████| 450kB 7.8MB/s 
     |████████████████████████████████| 870kB 10.6MB/s 
     |████████████████████████████████| 1.0MB 31.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=0a14070e6ad650787dc17ffcf423ad05d754bddd9b6d6abcaa3c67df0fce4ada
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


device: cuda


# Get data for part-of-speech tagging

In [6]:
train_sentences, train_labels, test_sentences, test_labels, _, _, label2index = get_pos_data("global-model-repr/probing", frac=0.01)
num_labels = len(label2index)
print("Training sentences:", len(train_sentences), "Test sentences:", len(test_sentences))
print("Unique labels:", num_labels)

Training sentences: 125 Test sentences: 21
Unique labels: 15
[tensor([ 9,  5,  9,  5,  7, 11,  4,  9,  9,  9,  5,  9,  5,  0, 11,  2,  0, 11,
         2,  0, 11,  2,  9,  5,  2,  0,  7, 11,  5]), tensor([ 5,  0, 11,  2,  0,  7, 11, 12, 12,  4, 13, 11,  2, 11, 10,  4,  5,  5]), tensor([ 9,  5,  7, 11,  4,  8, 13, 12,  4,  2,  1,  7, 11,  4,  2,  9,  5]), tensor([ 1,  2, 13, 12, 12,  4,  2,  1, 11,  2,  0,  9,  2,  0,  9,  5]), tensor([ 0,  9,  2,  9, 12,  7,  2,  0,  9,  9,  5,  3, 13, 12,  4,  8,  4,  9,
         9,  9,  3,  4,  2,  0,  7, 11, 11,  2,  0,  9, 11,  3,  2,  0, 11,  5]), tensor([ 0,  7, 12, 12,  4,  2,  0, 11,  2,  0, 11, 11,  5]), tensor([13,  4,  8, 13, 12,  4,  0, 11,  2, 13, 11, 11,  5]), tensor([ 0, 11, 12,  4,  2,  0,  9, 14,  9,  5,  9, 11,  2,  0, 11,  5]), tensor([ 8,  0, 11, 12,  3,  4, 10,  4, 11,  8,  4, 14,  4,  2, 11, 11,  5, 13,
        12, 10,  4,  0, 11,  8,  0,  9,  4, 10,  4,  0,  7, 11,  3,  7,  5]), tensor([13,  4, 13,  3,  3,  2,  0,  9, 11, 13, 12, 

# Set up model

In [0]:
class Classifier(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, input):
        output = self.linear(input)
        return output
    

class NonlinearClassifier(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(NonlinearClassifier, self).__init__()
        
        self.input2hidden = torch.nn.Linear(input_dim, input_dim)
        self.hidden2output = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, input):
        hidden = self.input2hidden(input)
        output = self.hidden2output(hidden)
        return output
    
    
def build_classifier(emb_dim, num_labels, nonlinear=False):

    if nonlinear:
        classifier = NonlinearClassifier(emb_dim, num_labels)
    else:
        classifier = Classifier(emb_dim, num_labels)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters())

    return classifier, criterion, optimizer


model_name = 'bert-base-cased'
# get model and tokenizer from Transformers
model, tokenizer, sep, emb_dim = get_model_and_tokenizer(model_name, device)
# build classifier
classifier, criterion, optimizer = build_classifier(emb_dim, num_labels)

In [4]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [0]:
print(classifier)

Classifier(
  (linear): Linear(in_features=768, out_features=15, bias=True)
)


# Train 

In [0]:
def train(num_epochs, train_sentences, train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer, layer=-1):
    
    num_total = sum([len(l) for l in train_labels])
    for i in range(num_epochs):
        total_loss = 0.
        num_correct = 0.
        for sentence, labels in zip(train_sentences, train_labels):
            optimizer.zero_grad()

            sentence_repr = get_sentence_repr(sentence, model, tokenizer, sep, model_name, device)
            # take layer representations
            sentence_repr = sentence_repr[layer]
            loss = 0
            for word_repr, label in zip(sentence_repr, labels):
                out = classifier(word_repr)
                # we'll just just a batch of size 1 for simplicity 
                out = torch.unsqueeze(out, 0)
                pred = out.max(1)[1]
                if pred == label.item():
                    num_correct += 1
                loss += criterion(out, label.unsqueeze(0))
            total_loss += loss.item()

            loss.backward()
            optimizer.step()
#         print('Training epoch: {}, loss: {}, accuracy: {}'.format(i, total_loss/num_total, num_correct/num_total))
    return total_loss/num_total, num_correct/num_total


# Evaluate

In [0]:
def evaluate(test_sentences, test_labels, 
             model, tokenizer, sep, model_name, device, 
             classifier, criterion, layer=-1):
    
    num_correct = 0.
    num_total = sum([len(l) for l in test_labels])
    total_loss = 0.
    with torch.no_grad():
        for sentence, labels in zip(test_sentences, test_labels):
            sentence_repr = get_sentence_repr(sentence, model, tokenizer, sep, model_name, device)
            sentence_repr = sentence_repr[layer]
            for word_repr, label in zip(sentence_repr, labels):
                out = classifier(word_repr)
                out = torch.unsqueeze(out, 0)
                pred = out.max(1)[1]
                if pred == label:
                    num_correct += 1
                total_loss += criterion(out, label.unsqueeze(0))

#     print('Testing loss: {}, accuracy: {}'.format(total_loss/num_total, num_correct/num_total))
    return total_loss/num_total, num_correct/num_total

# Experiment 1: Evaluate representation for POS quality

In [0]:
train_loss, train_accuracy = train(2, train_sentences, train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
test_loss, test_accuracy = evaluate(test_sentences, test_labels, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
print("Train accuracy: {}, Test accuracy: {}".format(train_accuracy, test_accuracy))

Train accuracy: 0.738976377952756, Test accuracy: 0.8425925925925926


# Experiment 2: Compare representation quality across layers

In [0]:
num_layers = 12
for l in range(num_layers):
    classifier, criterion, optimizer = build_classifier(emb_dim, num_labels)
    train_loss, train_accuracy = train(2, train_sentences, train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer, layer=l)
    test_loss, test_accuracy = evaluate(test_sentences, test_labels, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion, layer=l)
    print("layer: {}, test accuracy: {}".format(l, test_accuracy))

layer: 0, test accuracy: 0.8726851851851852
layer: 1, test accuracy: 0.9027777777777778
layer: 2, test accuracy: 0.9583333333333334
layer: 3, test accuracy: 0.9467592592592593
layer: 4, test accuracy: 0.9351851851851852
layer: 5, test accuracy: 0.9375
layer: 6, test accuracy: 0.9398148148148148
layer: 7, test accuracy: 0.9375
layer: 8, test accuracy: 0.9282407407407407
layer: 9, test accuracy: 0.9143518518518519
layer: 10, test accuracy: 0.9074074074074074
layer: 11, test accuracy: 0.9050925925925926


# Experiment 3: Non-linear classifier

In [0]:
num_layers = 12
for l in range(num_layers):
    classifier, criterion, optimizer = build_classifier(emb_dim, num_labels, nonlinear=True)
    train_loss, train_accuracy = train(2, train_sentences, train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer, layer=l)
    test_loss, test_accuracy = evaluate(test_sentences, test_labels, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion, layer=l)
    print("layer: {}, test accuracy: {}".format(l, test_accuracy))

layer: 0, test accuracy: 0.9004629629629629
layer: 1, test accuracy: 0.8518518518518519
layer: 2, test accuracy: 0.8935185185185185
layer: 3, test accuracy: 0.9282407407407407
layer: 4, test accuracy: 0.9513888888888888
layer: 5, test accuracy: 0.9490740740740741
layer: 6, test accuracy: 0.9513888888888888
layer: 7, test accuracy: 0.9699074074074074
layer: 8, test accuracy: 0.9699074074074074
layer: 9, test accuracy: 0.9722222222222222
layer: 10, test accuracy: 0.9328703703703703


# Experiment 4: Control labels

In this experiment we test to see how much of the good performance from Experiments 2 and 3 actually come from things the POS model learned, and how much of it just comes from the probe model. To test this, we use a method from Hewitt and Liang (https://arxiv.org/pdf/1909.03368.pdf). We make a <i>control task</i> which is unrelated to the POS task and do the same probing procedure on the control task. We then measure the <i>selectivity</i> of layers; the difference between their probed accuracy on the POS task and on the control task. If a layer has learned substantial things about the POS task in particular, it should be much better at the POS task than the control task; i.e. it should have high selectivity.

Following Hewitt and Liang, we use the following control task for POS tagging. Each word identity will be assigned a random POS tag, with the distribution of POS tags weighted according to their actual appearance. Each word identity will always have the same tag every time it appears. We then train and test the layers on predicting this tag from the embedding. Note that this tag is a deterministic function of the word identity, so high selectivity means the embedding actually has forgotten something about the word identity.



In [0]:
import random

vocabulary = set(
    word
      for sentence in (train_sentences + test_sentences)
      for word in sentence
)
all_labels = sum((x.tolist() for x in train_labels), [])
control_map = {word: random.choice(all_labels) for word in vocabulary}

control_train_labels = [torch.tensor([control_map[word] for word in sentence]) for sentence in train_sentences]
control_test_labels = [torch.tensor([control_map[word] for word in sentence]) for sentence in test_sentences]

In [27]:
num_layers = 12
for l in range(num_layers):
    classifier, criterion, optimizer = build_classifier(emb_dim, num_labels, nonlinear=True)
    train_loss, train_accuracy = train(2, train_sentences, control_train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer, layer=l)
    test_loss, test_accuracy = evaluate(test_sentences, control_test_labels, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion, layer=l)
    print("layer: {}, test accuracy: {}".format(l, test_accuracy))

layer: 0, test accuracy: 0.35648148148148145
layer: 1, test accuracy: 0.4050925925925926
layer: 2, test accuracy: 0.44212962962962965
layer: 3, test accuracy: 0.4699074074074074
layer: 4, test accuracy: 0.4837962962962963
layer: 5, test accuracy: 0.5
layer: 6, test accuracy: 0.48842592592592593
layer: 7, test accuracy: 0.5162037037037037
layer: 8, test accuracy: 0.4976851851851852
layer: 9, test accuracy: 0.5046296296296297
layer: 10, test accuracy: 0.4699074074074074
layer: 11, test accuracy: 0.4375
